##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fundamentos de personalização: tensores e operações

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/customization/basics"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/customization/basics.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/customization/basics.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/customization/basics.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Este é um tutorial introdutório do TensorFlow que mostra como:

- Importar o pacote necessário.
- Criar e usar tensores.
- Usar a aceleração de GPU.
- Criar um pipeline de dados com `tf.data.Dataset`.

## Importe o TensorFlow

Para começar, importe o módulo `tensorflow`. A partir do TensorFlow 2, a eager execution é ativada por padrão. A eager execution permite um front-end mais interativo para o TensorFlow, que você conhecerá depois em mais detalhes.

In [ ]:
import tensorflow as tf

## Tensores

Um Tensor é um array multidimensional. Semelhante aos objetos `ndarray` do NumPy, os objetos `tf.Tensor` têm um tipo de dados e um formato. Além disso, os `tf.Tensor`s podem residir na memória de um acelerador (como uma GPU). O TensorFlow oferece uma vasta biblioteca de operações (por exemplo, `tf.math.add`, `tf.linalg.matmul` e `tf.linalg.inv`), que consomem e produzem `tf.Tensor`s. Essas operações convertem automaticamente os tipos built-in do Python. Por exemplo:


In [ ]:
print(tf.math.add(1, 2))
print(tf.math.add([1, 2], [3, 4]))
print(tf.math.square(5))
print(tf.math.reduce_sum([1, 2, 3]))

# Operator overloading is also supported
print(tf.math.square(2) + tf.math.square(3))

Cada `tf.Tensor` tem um formato e um tipo de dados:

In [ ]:
x = tf.linalg.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

As diferenças mais óbvias entre arrays do NumPy e `tf.Tensor`s são as seguintes:

1. Os Tensores podem ser armazenados na memória de aceleradores (como GPU e TPU).
2. Os Tensores são imutáveis.

### Compatibilidade com o NumPy

A conversão entre um `tf.Tensor` do TensorFlow e um `ndarray` do NumPy é fácil:

- As operações do TensorFlow convertem automaticamente os ndarrays do NumPy para Tensores.
- As operações do NumPy convertem automaticamente os Tensores para ndarrays do NumPy.

Os Tensores são convertidos explicitamente para ndarrays do NumPy usando o método `.numpy()`. Essas conversões são geralmente baratas, já que o array e o `tf.Tensor` compartilham a representação de memória subjacente, se possível. No entanto, compartilhar a representação subjacente nem sempre é possível, já que o `tf.Tensor` pode ser hospedado na memória da GPU, enquanto os arrays do NumPy são sempre armazenados na memória do host, e a conversão envolve uma cópia da memória da GPU para a do host.

In [ ]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.math.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to NumPy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

## Aceleração de GPU

Várias operações do TensorFlow são aceleradas usando a GPU para computação. Sem qualquer anotação, o TensorFlow decide automaticamente usar a GPU ou CPU para uma operação — copiando o tensor entre a memória da CPU e da GPU, se necessário. Os Tensores produzidos por uma operação são geralmente armazenados na memória do dispositivo em que a operação está sendo executada. Por exemplo:

In [ ]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available: "),
print(tf.config.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

### Nomes do dispositivo

A propriedade `Tensor.device` fornece um nome de string totalmente qualificado do dispositivo que está hospedando o conteúdo do tensor. Esse nome codifica vários detalhes, como um identificador do endereço da rede do host em que esse programa está sendo executado e o dispositivo nesse host. Isso é necessário para a execução distribuída de um programa do TensorFlow. A string termina com `GPU:<N>` se o tensor estiver na GPU `N` do host.

### Posicionamento explícito do dispositivo

No TensorFlow, o *posicionamento* se refere à maneira como as operações individuais são atribuídas a (colocadas em) um dispositivo para execução. Conforme mencionado, quando nenhuma orientação explícita é fornecida, o TensorFlow decide automaticamente em que dispositivo executar uma operação e copia os tensores para esse dispositivo, se necessário.

No entanto, as operações do TensorFlow podem ser colocadas explicitamente em dispositivos específicos usando o gerenciador de contexto `tf.device`. Por exemplo:

In [ ]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.linalg.matmul(x, x)

  result = time.time()-start

  print("10 loops: {:0.2f}ms".format(1000*result))

# Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# Force execution on GPU #0 if available
if tf.config.list_physical_devices("GPU"):
  print("On GPU:")
  with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

## Datasets

Esta seção usa a API `tf.data.Dataset` na criação de um pipeline para alimentar seu modelo com dados. `tf.data.Dataset` é usado para criar pipelines de entrada complexos e eficientes a partir de partes simples e reutilizáveis que alimentarão os loops de treinamento ou avaliação do seu modelo. (Consulte o guia [tf.data: crie pipelines de entrada do TensorFlow](../../guide/data.ipynb) para saber mais.)

### Crie um `Dataset` de origem

Crie um dataset de *origem* usando uma das funções de fábrica, como `tf.data.Dataset.from_tensors` e `tf.data.Dataset.from_tensor_slices`, ou usando objetos que leem arquivos, como `tf.data.TextLineDataset` ou `tf.data.TFRecordDataset`. Consulte a seção *Lendo dados de entrada* do guia [tf.data: crie pipelines de entrada do TensorFlow](../../guide/data.ipynb) para mais informações.

In [ ]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

### Aplique transformações

Use as funções de transformação, como `tf.data.Dataset.map`, `tf.data.Dataset.batch` e `tf.data.Dataset.shuffle`, para aplicar transformações a registros de datasets.

In [ ]:
ds_tensors = ds_tensors.map(tf.math.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

### Faça a iteração

Os objetos `tf.data.Dataset` são compatíveis com a iteração para fazer loops nos registros:

In [ ]:
print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

print('\nElements in ds_file:')
for x in ds_file:
  print(x)